### Sensitivity analysis preparation

The CatBoost modeling notebook found these values to be the best model parameters:

```
{'iterations': 1538,
 'learning_rate': 0.052313318764570016,
 'depth': 5,
 'l2_leaf_reg': 0.02252067954779648,
 'loss_function': 'MAE',
 'random_seed': 42,
 'verbose': False}
```

In order to check model sensitivity a number of runs with different hyperparameter settings are needed. And all this needs to be 10 kfold cross validate all this! This can be very time consuming and resource intensive.

This notebook is a helper notebook that will do the runs to generate cross validated scores over a range of values preceding and succeeding the best model values, for each of these parameters:

```
'iterations': 1538
'learning_rate': 0.052313318764570016
'depth': 5
```

NOTE: in CatBoost, iterations means the number of trees.

We will hold all other hyperparameters constant at their best model values.

This will take a very long time to run. **Approximately 10 hours in some cases**


Because it can take so long it might be prone to crashing or not completing all runs. To avoid data loss we save the data to a text file (json) for parsing and sensitivity analysis in another notebook.

In [8]:
import pandas as pd 
import numpy as np
from catboost import CatBoostRegressor, Pool, metrics, cv
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

import altair as alt
import matplotlib.pyplot as plt
%matplotlib inline

import datetime
import json

import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 42

#### Data Preparation 

In [9]:
# Load vessel data
df_vessel = pd.read_csv('./data/cleansed/vessel_dwell_time.csv')

X = df_vessel.drop(columns=[ 'imo', 'vessel_name', 'time_seen', 'vessel_type', 'dwell_in_hr'])
y = df_vessel['dwell_in_hr']

In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1634 entries, 0 to 1633
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   target_terminal                   1634 non-null   object 
 1   avg_dwell_at_target_terminal      1634 non-null   float64
 2   num_of_vessel_at_target_terminal  1634 non-null   float64
 3   num_of_vessel_in_port             1634 non-null   float64
 4   weekday                           1634 non-null   int64  
 5   hour_of_day                       1634 non-null   int64  
 6   is_holiday                        1634 non-null   bool   
 7   vessel_operator                   1634 non-null   object 
 8   vessel_width                      1634 non-null   float64
 9   vessel_length                     1634 non-null   float64
 10  vessel_dwt                        1634 non-null   float64
dtypes: bool(1), float64(6), int64(2), object(2)
memory usage: 129.4+ KB


In [11]:
# create training set and testing set
# CatBoost has built-in support for categorical data. We are not required to handle it seperately
categorical_features_indices = [0,6,7]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

# create data pool for training set. Pool is specific to CatBoost
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_features_indices)
test_pool = Pool(data=X_test, label=y_test, cat_features=categorical_features_indices)

### Best model sanity check

Here we run a cross validation using the best model parameters we found in the CatBoost modeling phase of the project to reconfirm that best parameters that we should be using to do our sensitivity analysis.

In [13]:
%%time
best_model_params = {'iterations': 1538,
 'learning_rate': 0.052313318764570016,
 'depth': 5,
 'l2_leaf_reg': 0.02252067954779648,
 'loss_function': 'MAE',
 'random_seed': 42,
 'verbose': False}

cv_data = cv(
    pool=train_pool,
    params=best_model_params,   #best_model.get_params(), # get params from the best model 
    fold_count=10,
    logging_level='Silent'
)


CPU times: user 1min, sys: 20 s, total: 1min 20s
Wall time: 33.8 s


In [14]:
cv_data

,iterations,test-MAE-mean,test-MAE-std,train-MAE-mean,train-MAE-std
0,0,92.877398,18.473768,92.891975,2.110095
1,1,91.187773,18.402669,91.172426,2.034235
2,2,90.092463,18.378381,90.050731,1.964056
3,3,89.076065,18.372967,89.024624,1.943006
4,4,87.731904,18.356970,87.715147,1.894724
...,...,...,...,...,...
1533,1533,54.321525,11.543025,26.302379,0.861408
1534,1534,54.321185,11.543103,26.298282,0.861873
1535,1535,54.318501,11.541900,26.292712,0.861953
1536,1536,54.317159,11.541449,26.289147,0.861932


This confirms the best values from before in the CatBoost modeling notebook: 

* test-MAE-mean = 54.325687

In [30]:
cv_data.iloc[-1]

iterations        1537.000000
test-MAE-mean       54.325687
test-MAE-std        11.532876
train-MAE-mean      26.278719
train-MAE-std        0.856171
Name: 1537, dtype: float64

### Hyperparameter sensitivity

Now let's iterate over a range of values for the chosen hyperparameters. This will take a very long time since we're doing 10 fold cross validation for each of the combinations of values. It will take around 10 hours.

In [19]:
'''
Generate a linspace, but one that always crosses over and includes
a specific value as the center element of the linspace. Example:

    mycenter = 5
    x = get_centered_linspace(center = mycenter, 
                              steps = 3,
                              distance = 10,
                              use_int = False)

produces 'x' as follows:

    [0.0, 2.5, 5.0, 7.5, 10.0]

Notice center value is 5.0 (which is 'mycenter').
Notice the distance 'x[-1] - x[0]' is 10. 
Notice 'steps' is 3 steps in each direction from the 'mycenter' (self inclusive)

If use_int = True the result would have been:

    [0, 2, 5, 7, 10]

The length of the list returned is always odd.

'''
def get_centered_linspace(center, steps, distance, use_int):
    half_distance = distance/2
    left = center - half_distance
    right = center + half_distance
    left_segment = list(np.linspace(start=left, stop=center, num=steps, endpoint=True))
    right_segment = list(np.linspace(start=center, stop=right, num=steps))
    '''
    print(left, center, right, half_distance, steps)
    print(left_segment)
    print(right_segment)
    '''

    lin = list(left_segment[:-1] + right_segment)
    if use_int:
        return [int(i) for i in lin]
    return lin


'''
Get a human readable date time PDT to help in debug log data

'''
def get_now_datetime_str():
    return (datetime.datetime.now() - 
            datetime.timedelta(hours=7)).strftime("%Y-%m-%d %H:%M:%S PDT")

'''
Human readable date time PDT for file name

'''
def get_now_datetime_filename_str():
    return (datetime.datetime.now() - 
            datetime.timedelta(hours=7)).strftime("%Y-%m-%d_%H-%M-%S-PDT")


Set up our ranges to iterate over for each of our hyperparameter. The range will cross over the actual best model value for that hyperparameter. In fact for our tests, the middle value in the range will be the actual best model value for that parameter.

In [20]:
best_model_params = {'iterations': 1538,
 'learning_rate': 0.052313318764570016,
 'depth': 5,
 'l2_leaf_reg': 0.02252067954779648,
 'loss_function': 'MAE',
 'random_seed': 42,
 'verbose': False}

bmit = best_model_params['iterations']
bmlr = best_model_params['learning_rate']
bmd = best_model_params['depth']

In [21]:
best_model_params

{'iterations': 1538,
 'learning_rate': 0.052313318764570016,
 'depth': 5,
 'l2_leaf_reg': 0.02252067954779648,
 'loss_function': 'MAE',
 'random_seed': 42,
 'verbose': False}

### Example run

Prep the hyperparameter ranges that we will use for each cross-validation. 

NOTE: For our final report we will actually only use 1538 as the iteration to use. Because in our Feature Analysis we found that the two most important features by far were learning_rate and depth.

In [22]:
# this is the number of trees essentially, notice the center value 1538 is the actual best model value
iteration_range = get_centered_linspace(center=bmit, steps=5, distance=bmit*1, use_int=True)
iteration_range

[769, 961, 1153, 1345, 1538, 1730, 1922, 2114, 2307]

In [23]:
# notice the center value 0.052313318764570016 is the actual best model value
learning_rate_range = get_centered_linspace(center=bmlr, steps=5, distance=bmlr*1.5, use_int=False)
learning_rate_range

[0.013078329691142504,
 0.022887076959499382,
 0.03269582422785626,
 0.04250457149621314,
 0.052313318764570016,
 0.062122066032926894,
 0.07193081330128377,
 0.08173956056964066,
 0.09154830783799753]

In [29]:
# According to CatBoost documentation https://catboost.ai/en/docs/concepts/parameter-tuning
# "In most cases, the optimal depth ranges from 4 to 10. Values in the range
# from 6 to 10 are recommended." Also note the parameter is an int.
#
# In our case the best model value is actually 5. But since the docs are saying 
# optimal depth are from 4 to 10 anyway, let's just use that as the range.
#
depth_range = [4,5,6,7,8,9,10]
depth_range

[4, 5, 6, 7, 8, 9, 10]

Now do the actual work to looping over all the 3 hyperparameter ranges each time doing cross-validation to get the best MAE for that iteration.

The number of loops are 567 in this trial: 

* len(iteration_range) * len(learning_rate_range) * len(depth_range)

The hyperparameter values and resulting cross validated MAE will be saved to "sensitivity.txt" as it progresses. Since it takes a very long time to run so many loops, the system might crash and we don't want to lose hours worth of data.

Again this next cell take some time, **around 10 hours**.

In [ ]:
%%time

kfolds=2
i=0
total = len(iteration_range) * len(learning_rate_range) * len(depth_range)
log_file = '/content/drive/MyDrive/Colab Notebooks/data/696/sensitivity.{}.txt'.format(get_now_datetime_filename_str())
results = {
    'trial':{
        'starttime': get_now_datetime_str(),
        'iteration_range': iteration_range,
        'learning_rate_range': learning_rate_range,
        'depth_range': depth_range,
        'kfolds': kfolds
    },
    'runs':[]
}

for iterations in iteration_range:
    for learning_rate in learning_rate_range:
        for depth in depth_range:
            print('{}/{}, {}% - kfolds {}, iterations {}, learning_rate {}, depth {}'.format(i, total, round(((i/total)*100),2), kfolds, iterations, learning_rate, depth))
            hyper_params = best_model_params.copy()
            hyper_params['iterations'] = iterations
            hyper_params['learning_rate'] = learning_rate
            hyper_params['depth'] = depth

            cv_data = cv(
                pool=train_pool,
                params=hyper_params, # use our hyper parameters 
                fold_count=kfolds,
                logging_level='Silent'
            )

            result = {'hyper_params':hyper_params,
                      'cv_data':dict(cv_data.iloc[-1]),
                      'datetime': get_now_datetime_str()}
            results['runs'].append(result)

            with open(log_file, 'w') as sensitivity_file:
                sensitivity_file.write(json.dumps(results,indent=4))

            i+=1


0/567, 0.0% - kfolds 10, iterations 769, learning_rate 0.013078329691142504, depth 4
1/567, 0.18% - kfolds 10, iterations 769, learning_rate 0.013078329691142504, depth 5
2/567, 0.35% - kfolds 10, iterations 769, learning_rate 0.013078329691142504, depth 6
3/567, 0.53% - kfolds 10, iterations 769, learning_rate 0.013078329691142504, depth 7
4/567, 0.71% - kfolds 10, iterations 769, learning_rate 0.013078329691142504, depth 8
5/567, 0.88% - kfolds 10, iterations 769, learning_rate 0.013078329691142504, depth 9
6/567, 1.06% - kfolds 10, iterations 769, learning_rate 0.013078329691142504, depth 10
7/567, 1.23% - kfolds 10, iterations 769, learning_rate 0.022887076959499382, depth 4
8/567, 1.41% - kfolds 10, iterations 769, learning_rate 0.022887076959499382, depth 5
9/567, 1.59% - kfolds 10, iterations 769, learning_rate 0.022887076959499382, depth 6
10/567, 1.76% - kfolds 10, iterations 769, learning_rate 0.022887076959499382, depth 7
11/567, 1.94% - kfolds 10, iterations 769, learning_r

Now that the run completed, the data has been saved to "sensitivity.YYYY-mm-dd_H_M_S-PDT.txt" for more analysis and graphing. See other notebook for the graphing portion. (We want to keep this notebook for running long jobs.)

Quick sanity check to see the length of the results should be 567 which is the number of loops we had. And the final iteration of the loops shows that it in fact used the last element of the ranges: 

* iterations=2307
* learning_rate=0.09154830783799753
* depth=10

and the resulting test set MAE for those parameters was 

* 59.02130682452965

NOTE: for our report, the numbers will be rounded to 3 decimal places.

In [28]:
len(results)

567


In [27]:
results[-1]

{'hyper_params': {'iterations': 2307,
  'learning_rate': 0.09154830783799753,
  'depth': 10,
  'l2_leaf_reg': 0.02252067954779648,
  'loss_function': 'MAE',
  'random_seed': 42,
  'verbose': False},
 'cv_data': {'iterations': 2306.0,
  'test-MAE-mean': 59.02130682452965,
  'test-MAE-std': 10.864854692108924,
  'train-MAE-mean': 0.3213471632811678,
  'train-MAE-std': 0.07586881982866162},
 'datetime': '2022-10-17 14:30:10 PDT'}

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6b18b33d-3a56-4f49-ad6e-71ecea9f0183' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>